# Average Annual Household Transportation Costs
Out-of-pocket expenses including tolls, parking, and transit fares

These costs do not include other vehicle operating and purchase costs.

Costs are annualized using factor of 300, as used in the 2018 RTP.


In [1]:
import os, sys, shutil
sys.path.append(os.path.join(os.getcwd(),"inputs"))
sys.path.append(os.path.join(os.getcwd(),r"../../../scripts/summarize/standard"))
sys.path.append(os.path.join(os.getcwd(),r'../../..'))
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from shapely import wkt
# from input_configuration import base_year
# from emme_configuration import *
from standard_summary_configuration import *
pd.options.mode.chained_assignment = None
import os, sys, time, shutil
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.mode.chained_assignment = None
import h5py
# import toml
# config = toml.load(os.path.join(os.getcwd(),r'../../../configuration/input_configuration.toml'))


%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Relative path between notebooks and goruped output directories
relative_path = '../../../outputs/agg/dash'
survey_path = '../../../inputs/base_year/survey'

In [3]:
df = pd.read_csv(r'../../../outputs/agg/dash/person_cost.csv')
df_hh = pd.read_csv(r'..\..\..\outputs\agg\dash\hh_geog.csv')

annual_factor = 300

df['is_rgc'] = 0
df.loc[df['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_hh['is_rgc'] = 0
df_hh.loc[df_hh['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

In [4]:
df = df[(((df['mode'].isin(['SOV','HOV2','HOV3+']) & (df['dorp'] == 1)) 
          | (~df['mode'].isin(['SOV','HOV2','HOV3+']))))]

In [5]:
pd.options.display.float_format = '${:0,.0f}'.format

# County

In [6]:
_df = pd.DataFrame(df.groupby('hh_county').sum()['travcost_wt']/df_hh.groupby('hh_county').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df.loc['Region'] = df['travcost_wt'].sum()/df_hh['hhexpfac'].sum()*annual_factor
_df

,Average Annual Costs
hh_county,
King,"$2,191"
Kitsap,"$2,255"
Pierce,"$2,621"
Snohomish,"$2,946"
Region,"$2,421"


# Regional Growth Center

In [7]:
_df = pd.DataFrame(df.groupby('is_rgc').sum()['travcost_wt']/df_hh.groupby('is_rgc').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
is_rgc,
0,"$2,589"
1,$861


In [8]:
_df = pd.DataFrame(df.groupby('hh_rgc').sum()['travcost_wt']/df_hh.groupby('hh_rgc').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
hh_rgc,
Auburn,"$1,780"
Ballard-Interbay,$939
Bellevue,$828
Bothell Canyon Park,"$2,209"
Bremerton,$787
Burien,"$1,585"
Cascade,"$2,904"
Duwamish,"$1,867"
Everett,"$1,134"


# Regional Geography

In [9]:
_df = pd.DataFrame(df.groupby('hh_rg_proposed').sum()['travcost_wt']/df_hh.groupby('hh_rg_proposed').sum()['hhexpfac'])*annual_factor
_df.columns = ['Average Annual Costs']
_df

,Average Annual Costs
hh_rg_proposed,
CitiesTowns,"$3,256"
Core,"$2,359"
HCT,"$2,615"
Metro,"$1,518"
Rural,"$4,088"
UU,"$3,071"


# Equity Geography

In [10]:
results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'] > 0:
            _df = (df[df[equity_geog+geog_type] == 1].sum()['travcost_wt']/df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'])*annual_factor
            results_df.loc[equity_geog+geog_type,'costs'] = _df
        
inv_results_df = pd.DataFrame()
trip_rate_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']

# Calculate total work trips per each group
work_results_df = pd.DataFrame()
equity_geogs = ['hh_youth','hh_elderly','hh_english','hh_racial','hh_poverty','hh_disability']
for equity_geog in equity_geogs:
    for geog_type in ['_reg','_50']:
        if df_hh[df_hh[equity_geog+geog_type] == 1].sum()['hhexpfac'] > 0:
            _df = (df[df[equity_geog+geog_type] == 0].sum()['travcost_wt']/df_hh[df_hh[equity_geog+geog_type] == 0].sum()['hhexpfac'])*annual_factor
            inv_results_df.loc[equity_geog+geog_type,'costs'] = _df
        
# inv_results_df.index = ['NOT in '+i for i in results_df.index] 
_df = results_df.merge(inv_results_df, left_index=True, right_index=True)
_df.columns = ['Inside Equity Geog','Outside Equity Geog']
_df[~_df.isnull()]

,Inside Equity Geog,Outside Equity Geog
hh_youth_reg,"$2,869","$1,987"
hh_elderly_reg,"$2,496","$2,356"
hh_elderly_50,"$3,153","$2,420"
hh_english_reg,"$2,318","$2,482"
hh_racial_reg,"$2,176","$2,622"
hh_racial_50,"$2,224","$2,472"
hh_poverty_reg,"$2,171","$2,603"
hh_poverty_50,"$1,721","$2,439"
hh_disability_reg,"$2,390","$2,448"
